## **Naver Real Estate Crawling with Python Selenium**

- 준비물: Chrome, ChromeDriver, Selenium
- 참고  
https://velog.io/@kimdy0915/Selenium-%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC-%EC%9B%B9-%ED%8E%98%EC%9D%B4%EC%A7%80-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0%EC%84%B8%ED%8C%85

In [1]:
import json
import pandas as pd
import random
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from time import sleep
from tqdm import tqdm

import csv

In [2]:
# 드라이버 경로
driver_path = r'C:\Users\scsi\Desktop\오세훈\SCSI\2024-2\국토부\chromedriver-win64\chromedriver.exe'

# 드라이버 초기화
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

우선 사이트 접속, 지역 설정

In [ ]:
url = 'https://land.naver.com/'
driver.get(url)

# 지도로 보기 클릭
WebDriverWait(driver, 5).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "a.list"))
).click()

# 선택할 도시, 구/군, 동/리 변수
city = "서울시"
division = "서초구"
section = "서초동"

try:
    # 1. 도시 선택
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, f"//a[contains(@class, '_cityItm') and text()='{city}']"))
    ).click()
    print(f"도시 '{city}'가 선택되었습니다.")

    # 2. 구/군 선택
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, f"//a[contains(@class, '_dvsnItm') and text()='{division}']"))
    ).click()
    print(f"구/군 '{division}'가 선택되었습니다.")

    # 3. 동/리 선택 (normalize-space() 사용)
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, f"//a[contains(@class, '_secItm') and normalize-space(text())='{section}']"))
    ).click()
    print(f"동/리 '{section}'가 선택되었습니다.")

except TimeoutException as e:
    print(f"요소를 찾거나 클릭하는 데 실패했습니다: {str(e)}")

finally:
    # "확인 매물 보기" 버튼 클릭
    confirm_button = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'NPI=a: view') and text()='확인 매물 보기']"))
    )
    confirm_button.click()
    print("확인 매물 보기 버튼이 클릭되었습니다.")

클릭 가능한 버튼(=매물)을 찾고 리스트화

In [4]:
def extract_info(driver):
    try:
        # 데이터를 저장할 딕셔너리
        data = {}

        # 현재 페이지 HTML 가져오기
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        # 1. 제목 정보 추출
        try:
            title_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "info_title_wrap"))
            )
            title_text = title_element.find_element(By.CLASS_NAME, "info_title_name").text
            data['제목'] = title_text
        except Exception as e:
            print(f"제목 정보 추출 오류: {e}")

        # 2. 가격 정보 추출
        try:
            price_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "info_article_price"))
            )
            deal_type = price_element.find_element(By.CLASS_NAME, "type").text
            price = price_element.find_element(By.CLASS_NAME, "price").text
            data['거래 유형'] = deal_type
            data['가격'] = price
        except Exception as e:
            print(f"가격 정보 추출 오류: {e}")

        # 3. 테이블 정보 추출
        specific_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "info_table_wrap"))
        )
        specific_html = specific_element.get_attribute("outerHTML")
        soup = BeautifulSoup(specific_html, 'html.parser')

        info_table = soup.find('table', class_="info_table_wrap")
        if info_table:
            rows = info_table.find_all('tr', class_='info_table_item')
            for row in rows:
                headers = row.find_all('th', class_='table_th')
                values = row.find_all('td', class_='table_td')
                for header, value in zip(headers, values):
                    key = header.get_text(strip=True)
                    val = value.get_text(strip=True)

                    if key in ['중개사', '매물설명']:
                        continue

                    if key == '관리비':
                        val = val.replace('상세보기', '').strip()

                    data[key] = val

        # 4. 이미지 URL 추출
        try:
            # 이미지 요소를 기다린 후 가져오기
            image_element = WebDriverWait(driver, 1).until(
                EC.presence_of_element_located((By.CLASS_NAME, "floor_plan_img"))
            )
            # 이미지 URL 추출
            img_tag = image_element.find_element(By.TAG_NAME, "img")
            img_url = img_tag.get_attribute("src")
            
            # 딕셔너리에 저장
            data['이미지 URL'] = img_url
        except TimeoutException:
            # 평면도가 없을 경우 스킵하고 '이미지 URL'에 빈 값 저장
            data['이미지 URL'] = None
        except Exception as e:
            print(f"이미지 URL 추출 오류: {e}")

        return data

    except Exception as e:
        print(f"오류 발생: {e}")
        return {}

In [ ]:
button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "type6"))
)

# 버튼 클릭
button.click()
sleep(5)

# 전체 데이터를 저장할 딕셔너리
all_data = {}

try:
    # 초기 버튼 ID 리스트 생성
    valid_buttons = WebDriverWait(driver, 2).until(
        lambda d: [
            button for button in d.find_elements(By.CSS_SELECTOR, "a.marker_complex--apart[role='button']")
            if "is-small" not in button.get_attribute("class")
        ]
    )
    # 버튼 ID 리스트 생성
    button_ids = [button.get_attribute("id") or button.text for button in valid_buttons]

    # ID 리스트 순회 (tqdm 적용)
    for button_id in tqdm(button_ids, desc="Processing Buttons"):
        try:
            # ID로 버튼 찾기
            button = WebDriverWait(driver, 2).until(
                lambda d: d.find_element(By.CSS_SELECTOR, f"a[id='{button_id}']")
            )

            # 버튼 클릭 (JavaScript로 강제 클릭)
            driver.execute_script("arguments[0].click();", button)

            # 공인중개사협회 매물을 거르는 item_links 생성 로직
            try:
                # 모든 매물 요소 가져오기
                item_elements = WebDriverWait(driver, 10).until(
                    lambda d: d.find_elements(By.CSS_SELECTOR, "div.item")
                )

                # 공인중개사협회 매물과 네이버 보기 매물을 제외하고 item_link 리스트 생성
                item_links = []
                for item in item_elements:
                    # 1. 매물의 HTML 클래스 확인 (item_inner--agent)
                    if "item_inner--agent" in item.get_attribute("class"):
                        continue  # 공인중개사협회 매물은 건너뜀

                    # 2. label--cp 클래스 확인 (네이버 보기 매물 제외)
                    if item.find_elements(By.CSS_SELECTOR, "a.label--cp"):
                        continue  # 네이버 보기 매물은 건너뜀

                    # item 내의 링크 요소 찾기
                    try:
                        link = item.find_element(By.CSS_SELECTOR, "a.item_link")
                        item_links.append(link)
                    except Exception as e:
                        print(f"링크 요소를 찾는 중 오류 발생: {e}")
                        continue

                print(f"수집 가능한 매물 수: {len(item_links)}")

            except Exception as e:
                print("item_links가 비어 있어 작업을 건너뜁니다.")

            # 링크에서 정보 추출 및 딕셔너리에 저장
            if not item_links:
                continue

            # "단지정보" 버튼 클릭
            wait = WebDriverWait(driver, 10)
            wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), '단지정보') and contains(@class, 'complex_link')]"))).click()

            # 제목 전체를 포함하는 XPath로 주소 추출
            address = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "address"))
            ).text

            for idx, link in enumerate(item_links):
                try:
                    sleep(0.5)
                    # 링크 클릭
                    link.click()

                    # 정보 추출
                    data = extract_info(driver)

                    # 주소 값을 추가
                    data['주소'] = address

                    # 딕셔너리에 저장 (고유 키로 인덱스 사용)
                    key = f"item_{len(all_data) + 1}"
                    all_data[key] = data

                except Exception as e:
                    print(f"오류 발생 (item_{len(all_data) + 1}): {e}")
                    continue


            # "상세페이지 닫기" 버튼 클릭
            WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable((By.XPATH, "//button[@class='btn_close' and @aria-label='상세페이지 닫기']"))
            ).click()

        except Exception as e:
            print(f"버튼 {button_id} skip.")
            continue

except Exception as e:
    print(f"초기 버튼 목록 생성 중 오류 발생: {e}")

# 최종 데이터 출력
print("모든 작업이 완료되었습니다.")
print("전체 데이터:", all_data)

In [ ]:
# CSV 파일 저장 함수
def save_to_csv(data, file_name="output.csv"):
    if not data:
        print("저장할 데이터가 없습니다.")
        return

    # 모든 아이템의 키들의 합집합 구하기
    keys = set()
    for item in data.values():
        keys.update(item.keys())
    keys = sorted(keys)  # 키를 정렬하여 일관성 유지

    # CSV 파일 저장
    with open(file_name, mode="w", newline="", encoding="utf-8-sig") as file:  # utf-8-sig 사용
        writer = csv.DictWriter(file, fieldnames=keys)

        # 헤더 작성
        writer.writeheader()

        # 데이터 작성
        for item in data.values():
            # 없는 키는 '-'로 채우기
            row = {key: item.get(key, "-") for key in keys}
            writer.writerow(row)

    print(f"데이터가 '{file_name}' 파일로 저장되었습니다.")

# CSV로 저장
save_to_csv(all_data, file_name=f"real_state_data_{section}.csv")